[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/TotoroUI-jupyter/blob/main/test.ipynb)

In [ ]:
!pip install -q torchsde einops diffusers accelerate xformers==0.0.25
!apt -y install -qq aria2

%cd /content
!git clone -b totoro https://github.com/camenduru/ComfyUI /content/TotoroUI
%cd /content/TotoroUI

import torch
import numpy as np
from PIL import Image
import totoro
import nodes

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/RunDiffusion/Juggernaut-XL-v9/resolve/main/Juggernaut-XL_v9_RunDiffusionPhoto_v2.safetensors -d /content/TotoroUI/model -o Juggernaut-XL_v9_RunDiffusionPhoto_v2.safetensors

with torch.no_grad():
  model_patcher, clip, vae, clipvision = totoro.sd.load_checkpoint_guess_config("/content/TotoroUI/model/Juggernaut-XL_v9_RunDiffusionPhoto_v2.safetensors", output_vae=True, output_clip=True, embedding_directory=None)

In [ ]:
latent = {"samples":torch.zeros([1, 4, 1024 // 8, 1024 // 8])}

tokens = clip.tokenize("toy car")
cond, pooled = clip.encode_from_tokens(tokens, return_pooled=True)
cond = [[cond, {"pooled_output": pooled}]]

n_tokens = clip.tokenize("red")
n_cond, n_pooled = clip.encode_from_tokens(n_tokens, return_pooled=True)
n_cond = [[n_cond, {"pooled_output": n_pooled}]]

sample = nodes.common_ksampler(model=model_patcher, 
                          seed=5454, 
                          steps=20, 
                          cfg=8.0, 
                          sampler_name="euler", 
                          scheduler="normal", 
                          positive=cond, 
                          negative=n_cond,
                          latent=latent, 
                          denoise=1, 
                          disable_noise=False, 
                          start_step=0, 
                          last_step=20, 
                          force_full_denoise=True)

with torch.inference_mode():
  sample = sample[0]["samples"].to(torch.float16)
  vae.first_stage_model.cuda()
  decoded = vae.decode_tiled(sample).detach()

Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0])